In [1]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

In [3]:
import src.data as data
import src.utility as utility

In [4]:
mode = 'test'
steps_behind_event=4
steps_after_event=3

In [5]:
#sensors = data.sensors()
s = data.speeds_original(mode) #.merge(sensors, how='left')
# - events
e = data.events(mode)
# - weather
# ......

# join dataframes
joined_df = utility.merge_speed_events(s, e)

caching resources/dataset/originals/speeds_train.csv.gz
caching resources/dataset/preprocessed/events_train.csv.gz


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [6]:
joined_df[(joined_df.KEY_2 == '0_333') & (joined_df.DATETIME_UTC == '2018-09-05 18:00:00') ]

,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2,event_index,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KM_EVENT,step_duration
75111,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
75112,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
75113,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
75114,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
75115,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
dioc[(dioc.KEY_2 == '0_333') & (dioc.DATETIME_UTC == '2018-09-05 18:00:00') ]

,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2,index,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KM_EVENT,step_duration
75111,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,15854.0,2018-08-02 07:45:00,2018-10-03 10:15:00,48.0,Pavimentazione,498.0,491.0,496.0,5963.0
75112,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,16477.0,2018-08-27 07:45:00,2018-09-07 15:30:00,1.0,Pavimentazione,415.0,412.0,NaN,1088.0
75113,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,16703.0,2018-09-03 08:45:00,2018-09-08 10:15:00,1.0,Pavimentazione,394.0,391.0,NaN,487.0
75114,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,16769.0,2018-09-04 12:45:00,2018-09-07 14:45:00,29.0,Opera_arte,363.0,361.0,NaN,297.0
75115,0,2018-09-05 18:00:00,333.0,216.837609,38.239321,79.282,459.512,695.0,0_333,34645.0,2018-09-05 09:00:00,2018-09-05 18:00:00,55.0,Opera_arte,515.0,508.0,513.0,37.0


In [24]:
s[(s.KEY_2 == '0_333') & (s.DATETIME_UTC == '2018-09-05 18:00:00') ]

,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2
5959369,0,2018-09-05 18:00:00,333,216.837609,38.239321,79.282,459.512,695,0_333


In [25]:
def time_windows_event(dataset_df, steps_behind=10, steps_after=3, step=15*60):
    """ Filter the dataset to get a window containing n time steps before the beginning
        of the event and m time steps after the end for each involved sensor

        dataset_df (df): dataset dataframe
        steps_behind (int): n (not including the event start)
        steps_after (int): m (not including the event start)
    """
    tqdm.pandas()
    # get the first time step of each event for each sensor
    start_events = dataset_df[dataset_df.event_index.notnull()]
    start_events = start_events[['KEY_2','event_index','DATETIME_UTC']].groupby(['KEY_2','event_index'], as_index=False).min()
    start_events['sample_id'] = start_events.index
    print('Total events found:', start_events.shape[0])

    start_delta = datetime.timedelta(seconds=step*steps_behind)
    end_delta = datetime.timedelta(seconds=step*steps_after)

    # construct the time window for each event beginning
    start_events['window'] = start_events.progress_apply(lambda x:
                                    list(pd.date_range(start=x.DATETIME_UTC - start_delta,
                                                       end=x.DATETIME_UTC + end_delta,
                                                       freq=f'{step}s')), axis=1)
    start_events = start_events.drop('DATETIME_UTC', axis=1)

    # build the filter
    filter_df = pd.DataFrame({ col: np.repeat(start_events[col].values, start_events['window'].str.len())
        for col in start_events.columns.drop('window')
    }).assign(**{'DATETIME_UTC': np.concatenate(start_events['window'].values)})
    print('Filter size:', filter_df.shape[0])
    #return filter_df

    # join to filter the desired rows, removing duplicated road-timestamps from the dataset (they will be duplicated again
    # after the merge, since a different time window is created for each event)
    
    return filter_df.merge(data.speeds_original(), how='left', on=['KEY_2','DATETIME_UTC']) \
            .sort_values(['sample_id','DATETIME_UTC'])

In [11]:
filter_df = time_windows_event(joined_df, steps_behind=steps_behind_event, steps_after=steps_after_event)

Total events found: 8117



Filter size: 64936


In [29]:
filter_df.head(12) #[(filter_df.KEY_2 == '0_333') & (filter_df.DATETIME_UTC == '2018-09-05 17:15:00')]

,KEY_2,event_index,sample_id,DATETIME_UTC
0,0_333,16789.0,0,2018-09-05 17:15:00
1,0_333,16789.0,0,2018-09-05 17:30:00
2,0_333,16789.0,0,2018-09-05 17:45:00
3,0_333,16789.0,0,2018-09-05 18:00:00
4,0_333,16789.0,0,2018-09-05 18:15:00
5,0_333,16789.0,0,2018-09-05 18:30:00
6,0_333,16789.0,0,2018-09-05 18:45:00
7,0_333,16789.0,0,2018-09-05 19:00:00
8,0_333,17038.0,1,2018-09-15 10:15:00
9,0_333,17038.0,1,2018-09-15 10:30:00


In [22]:
filter_df.shape

(64936, 4)

In [24]:
joined_df[(joined_df.KEY_2 == '0_333') & (joined_df.DATETIME_UTC == '2018-09-05 17:15:00')]

,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2,event_index,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KM_EVENT,step_duration
540,0,2018-09-05 17:15:00,333.0,218.159601,36.571273,64.72,432.006,736.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
541,0,2018-09-05 17:15:00,333.0,218.159601,36.571273,64.72,432.006,736.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
542,0,2018-09-05 17:15:00,333.0,218.159601,36.571273,64.72,432.006,736.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
543,0,2018-09-05 17:15:00,333.0,218.159601,36.571273,64.72,432.006,736.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
544,0,2018-09-05 17:15:00,333.0,218.159601,36.571273,64.72,432.006,736.0,0_333,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
sensors = data.sensors()
weather = data.weather()
    
print('Merging speeds and events...')
e = data.events(mode)
se = utility.merge_speed_events(data.speeds_original(mode), e)

print('Done')

# create the time windows for each event
print('Creating time windows for events...')
joined_df = utility.time_windows_event(se, mode=mode, steps_behind=steps_behind_event, steps_after=steps_after_event)

# add other dataframes
# - events
#joined_df = joined_df.merge(e.drop(['']), how='left', left_on='event_index', right_index=True)

caching sensors

sensors loaded in: 0.0155 s

shape of the dataframe is: (2171, 5)
caching resources/dataset/preprocessed/base_structure_df_weather.csv.gz
Merging speeds and events...
caching resources/dataset/preprocessed/events_test.csv.gz
caching resources/dataset/originals/speeds_test.csv.gz
Done
Creating time windows for events...
Total events found: 3449



Filter size: 27592


In [6]:
joined_df.head()

,KEY_2,event_index,sample_id,DATETIME_UTC,KEY,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES
0,0_333,4244.0,0,2018-12-07 10:00:00,0.0,333.0,241.072538,21.877692,195.778,304.184,171.0
1,0_333,4244.0,0,2018-12-07 10:15:00,0.0,333.0,235.650724,21.686928,184.452,305.802,199.0
2,0_333,4244.0,0,2018-12-07 10:30:00,0.0,333.0,232.699861,24.707300,158.564,310.656,216.0
3,0_333,4244.0,0,2018-12-07 10:45:00,0.0,333.0,230.531447,22.642944,186.070,331.690,217.0
4,0_333,4244.0,0,2018-12-07 11:00:00,0.0,333.0,237.263172,25.334048,177.980,304.184,186.0


In [7]:
joined_df.shape

(27592, 11)

In [8]:
joined_df = joined_df.merge(data.events_original().drop(['KEY','KEY_2'],axis=1), how='left', left_on='event_index', right_index=True)

caching resources/dataset/originals/events_train.csv.gz


In [9]:
# - weather
# - sensors
joined_df = joined_df.merge(weather, how='left').merge(sensors, how='left').round(4).groupby('sample_id').agg({
    'KEY':'first',
    'KM':'first',
    'event_index':lambda x: x.values[steps_behind_event],
    'DATETIME_UTC':list,
    'SPEED_AVG':list, #[list, lambda x: x[0:event_beginning_step].dropna().mean()],
    'SPEED_SD':list,
    'SPEED_MAX':list,
    'SPEED_MIN':list,
    'N_VEHICLES':list,
    'EMERGENCY_LANE':'first',
    'LANES':'first',
    'ROAD_TYPE':'first',
    'EVENT_DETAIL':lambda x: x.values[steps_behind_event],
    'EVENT_TYPE':lambda x: x.values[steps_behind_event],
    'WEATHER': list,
    'DISTANCE': list,
    'TEMPERATURE': list,
    'MIN_TEMPERATURE': list,
    'MAX_TEMPERATURE': list
})

In [11]:
# split the last m measures in different columns
def split_prediction_fields(row, event_beginning_step):
    return pd.Series((
            row.DATETIME_UTC[:event_beginning_step], row.DATETIME_UTC[event_beginning_step:], 
            row.SPEED_AVG[:event_beginning_step],    row.SPEED_AVG[event_beginning_step:],
            row.SPEED_SD[:event_beginning_step],
            row.SPEED_MAX[:event_beginning_step],
            row.SPEED_MIN[:event_beginning_step],
            row.N_VEHICLES[:event_beginning_step],
            row.WEATHER[:event_beginning_step],
            row.DISTANCE[:event_beginning_step],
            row.TEMPERATURE[:event_beginning_step],
            row.MIN_TEMPERATURE[:event_beginning_step],
            row.MAX_TEMPERATURE[:event_beginning_step],
    ))

print('Splitting time steps into separate columns...')

columns_to_split = ['DATETIME_UTC','DATETIME_UTC_y',
                    'SPEED_AVG','SPEED_AVG_Y',
                    'SPEED_SD', 'SPEED_MAX', 'SPEED_MIN', 'N_VEHICLES', 'WEATHER', 'DISTANCE',
                    'TEMPERATURE', 'MIN_TEMPERATURE', 'MAX_TEMPERATURE']
joined_df[columns_to_split] = joined_df.apply(split_prediction_fields, axis=1, event_beginning_step=steps_behind_event)

for col_name in columns_to_split:
    if col_name.upper().endswith('_Y'):
        new_cols = ['{}_{}'.format(col_name, i) for i in range(0, steps_after_event+1)]
    else:
        new_cols = ['{}_{}'.format(col_name, i) for i in range(-steps_behind_event, 0)]

    joined_df[new_cols] = pd.DataFrame(joined_df[col_name].values.tolist(), index=joined_df.index)

Splitting time steps into separate columns...


In [14]:
joined_df.head(3)

,KEY,KM,event_index,DATETIME_UTC,SPEED_AVG,SPEED_SD,SPEED_MAX,SPEED_MIN,N_VEHICLES,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,WEATHER,DISTANCE,TEMPERATURE,MIN_TEMPERATURE,MAX_TEMPERATURE,DATETIME_UTC_y,SPEED_AVG_Y,DATETIME_UTC_-4,DATETIME_UTC_-3,DATETIME_UTC_-2,DATETIME_UTC_-1,DATETIME_UTC_y_0,DATETIME_UTC_y_1,DATETIME_UTC_y_2,DATETIME_UTC_y_3,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_AVG_Y_0,SPEED_AVG_Y_1,SPEED_AVG_Y_2,SPEED_AVG_Y_3,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1,TEMPERATURE_-4,TEMPERATURE_-3,TEMPERATURE_-2,TEMPERATURE_-1,MIN_TEMPERATURE_-4,MIN_TEMPERATURE_-3,MIN_TEMPERATURE_-2,MIN_TEMPERATURE_-1,MAX_TEMPERATURE_-4,MAX_TEMPERATURE_-3,MAX_TEMPERATURE_-2,MAX_TEMPERATURE_-1
sample_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,333.0,4244.0,"[2018-12-07 10:00:00, 2018-12-07 10:15:00, 2018-12-07 10:30:00, 2018-12-07 10:45:00]","[241.0725, 235.6507, 232.6999, 230.5314]","[21.8777, 21.6869, 24.7073, 22.6429]","[304.184, 305.802, 310.656, 331.69]","[195.778, 184.452, 158.564, 186.07]","[171.0, 199.0, 216.0, 217.0]",1.0,2.0,1.0,20.0,extended_accident,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","[nan, nan, nan, nan]","[nan, nan, nan, nan]","[nan, nan, nan, nan]","[2018-12-07 11:00:00, 2018-12-07 11:15:00, 2018-12-07 11:30:00, 2018-12-07 11:45:00]","[237.2632, 240.5375, 237.6489, 242.4303]",2018-12-07 10:00:00,2018-12-07 10:15:00,2018-12-07 10:30:00,2018-12-07 10:45:00,2018-12-07 11:00:00,2018-12-07 11:15:00,2018-12-07 11:30:00,2018-12-07 11:45:00,241.0725,235.6507,232.6999,230.5314,237.2632,240.5375,237.6489,242.4303,21.8777,21.6869,24.7073,22.6429,304.184,305.802,310.656,331.690,195.778,184.452,158.564,186.070,171.0,199.0,216.0,217.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,337.0,4155.0,"[2018-12-03 11:30:00, 2018-12-03 11:45:00, 2018-12-03 12:00:00, 2018-12-03 12:15:00]","[227.0271, 221.5783, 223.0587, 224.6654]","[32.3873, 33.183, 33.4944, 34.4274]","[309.038, 341.398, 323.6, 304.184]","[59.866, 61.484, 61.484, 64.72]","[201.0, 203.0, 237.0, 212.0]",1.0,2.0,1.0,25.0,Calamita_naturale,"[Quasi Sereno, Quasi Sereno, Quasi Sereno, Quasi Sereno]","[226.0, 226.0, 226.0, 226.0]","[17.0, 17.0, 17.0, 17.0]","[8.0, 8.0, 8.0, 8.0]","[17.0, 17.0, 17.0, 17.0]","[2018-12-03 12:30:00, 2018-12-03 12:45:00, 2018-12-03 13:00:00, 2018-12-03 13:15:00]","[224.4166, 225.7645, 224.8808, 222.0327]",2018-12-03 11:30:00,2018-12-03 11:45:00,2018-12-03 12:00:00,2018-12-03 12:15:00,2018-12-03 12:30:00,2018-12-03 12:45:00,2018-12-03 13:00:00,2018-12-03 13:15:00,227.0271,221.5783,223.0587,224.6654,224.4166,225.7645,224.8808,222.0327,32.3873,33.1830,33.4944,34.4274,309.038,341.398,323.600,304.184,59.866,61.484,61.484,64.720,201.0,203.0,237.0,212.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,226.0,226.0,226.0,226.0,17.0,17.0,17.0,17.0,8.0,8.0,8.0,8.0,17.0,17.0,17.0,17.0
2,0.0,337.0,4244.0,"[2018-12-07 10:00:00, 2018-12-07 10:15:00, 2018-12-07 10:30:00, 2018-12-07 10:45:00]","[220.773, 222.0066, 212.8068, 220.1442]","[36.406, 34.5186, 27.6025, 31.87]","[321.982, 351.106, 297.712, 326.836]","[59.866, 63.102, 74.428, 63.102]","[212.0, 228.0, 244.0, 286.0]",1.0,2.0,1.0,20.0,extended_accident,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","[nan, nan, nan, nan]","[nan, nan, nan, nan]","[nan, nan, nan, nan]","[2018-12-07 11:00:00, 2018-12-07 11:15:00, 2018-12-07 11:30:00, 2018-12-07 11:45:00]","[215.485, 222.5566, 222.8882, 222.6499]",2018-12-07 10:00:00,2018-12-07 10:15:00,2018-12-07 10:30:00,2018-12-07 10:45:00,2018-12-07 11:00:00,2018-12-07 11:15:00,2018-12-07 11:30:00,2018-12-07 11:45:00,220.7730,222.0066,212.8068,220.1442,215.4850,222.5566,222.8882,222.6499,36.4060,34.5186,27.6025,31.8700

In [15]:
joined_df = joined_df.drop(columns_to_split, axis=1)

In [16]:
joined_df.head(3)

,KEY,KM,event_index,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,DATETIME_UTC_-4,DATETIME_UTC_-3,DATETIME_UTC_-2,DATETIME_UTC_-1,DATETIME_UTC_y_0,DATETIME_UTC_y_1,DATETIME_UTC_y_2,DATETIME_UTC_y_3,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_AVG_Y_0,SPEED_AVG_Y_1,SPEED_AVG_Y_2,SPEED_AVG_Y_3,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1,TEMPERATURE_-4,TEMPERATURE_-3,TEMPERATURE_-2,TEMPERATURE_-1,MIN_TEMPERATURE_-4,MIN_TEMPERATURE_-3,MIN_TEMPERATURE_-2,MIN_TEMPERATURE_-1,MAX_TEMPERATURE_-4,MAX_TEMPERATURE_-3,MAX_TEMPERATURE_-2,MAX_TEMPERATURE_-1
sample_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,333.0,4244.0,1.0,2.0,1.0,20.0,extended_accident,2018-12-07 10:00:00,2018-12-07 10:15:00,2018-12-07 10:30:00,2018-12-07 10:45:00,2018-12-07 11:00:00,2018-12-07 11:15:00,2018-12-07 11:30:00,2018-12-07 11:45:00,241.0725,235.6507,232.6999,230.5314,237.2632,240.5375,237.6489,242.4303,21.8777,21.6869,24.7073,22.6429,304.184,305.802,310.656,331.690,195.778,184.452,158.564,186.070,171.0,199.0,216.0,217.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,337.0,4155.0,1.0,2.0,1.0,25.0,Calamita_naturale,2018-12-03 11:30:00,2018-12-03 11:45:00,2018-12-03 12:00:00,2018-12-03 12:15:00,2018-12-03 12:30:00,2018-12-03 12:45:00,2018-12-03 13:00:00,2018-12-03 13:15:00,227.0271,221.5783,223.0587,224.6654,224.4166,225.7645,224.8808,222.0327,32.3873,33.1830,33.4944,34.4274,309.038,341.398,323.600,304.184,59.866,61.484,61.484,64.720,201.0,203.0,237.0,212.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,226.0,226.0,226.0,226.0,17.0,17.0,17.0,17.0,8.0,8.0,8.0,8.0,17.0,17.0,17.0,17.0
2,0.0,337.0,4244.0,1.0,2.0,1.0,20.0,extended_accident,2018-12-07 10:00:00,2018-12-07 10:15:00,2018-12-07 10:30:00,2018-12-07 10:45:00,2018-12-07 11:00:00,2018-12-07 11:15:00,2018-12-07 11:30:00,2018-12-07 11:45:00,220.7730,222.0066,212.8068,220.1442,215.4850,222.5566,222.8882,222.6499,36.4060,34.5186,27.6025,31.8700,321.982,351.106,297.712,326.836,59.866,63.102,74.428,63.102,212.0,228.0,244.0,286.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
data.events_original('test').drop(['KEY','KEY_2'],axis=1).head()

caching resources/dataset/originals/events_test.csv.gz


,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START
0,2018-08-09 07:55:47,2018-12-21 18:12:56,3.0,Opera_arte,449,448
1,2018-09-25 19:23:56,2018-12-21 07:49:20,3.0,Opera_arte,2148,2148
2,2018-10-22 15:14:36,2018-12-21 08:08:30,15.0,Pavimentazione,267,267
3,2018-11-08 10:13:08,2018-12-21 07:52:19,0.0,Gestione_viabilita,2843,2844
4,2018-11-20 08:21:57,2018-12-07 08:51:39,0.0,Gestione_viabilita,1816,1821


In [32]:
data.events('test').drop(['KEY','KEY_2','DATETIME_UTC','step_duration','START_DATETIME_UTC','END_DATETIME_UTC'],axis=1).groupby(['index']).first()

,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KM_EVENT
index,,,,,,,
0,2018-08-09 08:00:00,2018-12-21 18:15:00,3.0,Opera_arte,449,448,NaN
1,2018-09-25 19:30:00,2018-12-21 08:00:00,3.0,Opera_arte,2150,2143,2148.0
2,2018-10-22 15:15:00,2018-12-21 08:15:00,15.0,Pavimentazione,269,262,267.0
3,2018-11-08 10:15:00,2018-12-21 08:00:00,0.0,Gestione_viabilita,2844,2843,NaN
4,2018-11-20 08:30:00,2018-12-07 09:00:00,0.0,Gestione_viabilita,1821,1816,NaN
5,2018-11-27 17:15:00,2018-12-10 08:45:00,0.0,Gestione_viabilita,2826,2819,2824.0
6,2018-11-27 17:15:00,2018-12-12 18:45:00,0.0,Gestione_viabilita,2422,2415,2420.0
7,2018-11-29 09:15:00,2018-12-07 19:00:00,0.0,Gestione_viabilita,2518,2517,NaN
8,2018-11-29 17:30:00,2018-12-11 16:00:00,0.0,Gestione_viabilita,297,295,NaN


## Check if speeds are correct

In [4]:
dataset = data.base_dataset()

caching base dataset train


In [5]:
dataset.head(10)

,KEY,KM,event_index,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,DATETIME_UTC_-4,DATETIME_UTC_-3,DATETIME_UTC_-2,DATETIME_UTC_-1,DATETIME_UTC_y_0,DATETIME_UTC_y_1,DATETIME_UTC_y_2,DATETIME_UTC_y_3,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_AVG_Y_0,SPEED_AVG_Y_1,SPEED_AVG_Y_2,SPEED_AVG_Y_3,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1,TEMPERATURE_-4,TEMPERATURE_-3,TEMPERATURE_-2,TEMPERATURE_-1,MIN_TEMPERATURE_-4,MIN_TEMPERATURE_-3,MIN_TEMPERATURE_-2,MIN_TEMPERATURE_-1,MAX_TEMPERATURE_-4,MAX_TEMPERATURE_-3,MAX_TEMPERATURE_-2,MAX_TEMPERATURE_-1
0,0.0,333.0,16789.0,1,2,1,11,Veicolo_in_avaria,2018-09-05 17:15:00,2018-09-05 17:30:00,2018-09-05 17:45:00,2018-09-05 18:00:00,2018-09-05 18:15:00,2018-09-05 18:30:00,2018-09-05 18:45:00,2018-09-05 19:00:00,218.1596,215.1602,221.5288,216.8376,212.2848,213.7758,222.5046,216.1972,36.5713,34.3015,38.7876,38.2393,432.006,449.804,465.984,459.512,64.720,71.192,69.574,79.282,736.0,767.0,731.0,695.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,33.0,33.0,33.0,33.0,29.0,29.0,29.0,29.0,21.0,21.0,21.0,21.0,29.0,29.0,29.0,29.0
1,0.0,333.0,17038.0,1,2,1,4,Ostacolo_in_carreggiata,2018-09-15 10:15:00,2018-09-15 10:30:00,2018-09-15 10:45:00,2018-09-15 11:00:00,2018-09-15 11:15:00,2018-09-15 11:30:00,2018-09-15 11:45:00,2018-09-15 12:00:00,223.7823,220.6725,217.7750,224.6205,223.4790,227.4011,224.2504,228.7991,34.9964,32.4112,33.5583,33.8886,467.602,389.938,404.500,394.792,61.484,74.428,61.484,74.428,539.0,557.0,583.0,546.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,221.0,221.0,221.0,221.0,26.0,26.0,26.0,26.0,19.0,19.0,19.0,19.0,24.0,24.0,24.0,24.0
2,0.0,333.0,17088.0,1,2,1,4,Ostacolo_in_carreggiata,2018-09-17 15:15:00,2018-09-17 15:30:00,2018-09-17 15:45:00,2018-09-17 16:00:00,2018-09-17 16:15:00,2018-09-17 16:30:00,2018-09-17 16:45:00,2018-09-17 17:00:00,226.0311,220.6224,221.8755,222.3986,221.7808,225.0752,224.7348,219.9737,37.8230,35.9977,41.7747,34.4392,464.366,417.444,457.894,430.388,63.102,64.720,61.484,64.720,609.0,569.0,610.0,614.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,33.0,33.0,33.0,33.0,27.0,27.0,27.0,27.0,21.0,21.0,21.0,21.0,27.0,27.0,27.0,27.0
3,0.0,333.0,17144.0,1,2,1,18,Veicolo_in_avaria,2018-09-19 07:30:00,2018-09-19 07:45:00,2018-09-19 08:00:00,2018-09-19 08:15:00,2018-09-19 08:30:00,2018-09-19 08:45:00,2018-09-19 09:00:00,2018-09-19 09:15:00,224.5611,226.4205,223.1798,224.3371,223.9283,218.6939,220.7767,224.3234,41.8797,36.9565,36.9518,35.4471,469.220,465.984,440.096,462.748,67.956,74.428,66.338,61.484,655.0,618.0,590.0,633.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,33.0,33.0,33.0,33.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,26.0,26.0,26.0,26.0
4,0.0,333.0,17370.0,1,2,1,4,Ostacolo_in_carreggiata,2018-09-27 15:00:00,2018-09-27 15:15:00,2018-09-27 15:30:00,2018-09-27 15:45:00,2018-09-27 16:00:00,2018-09-27 16:15:00,2018-09-27 16:30:00,2018-09-27 16:45:00,241.0884,239.9227,230.4686,236.7613,235.4108,234.6779,233.1017,230.3678,20.3806,21.8812,19.9869,20.8277,328.454,321.982,286.386,325.218,190.924,192.542,186.070,184.452,253.0,261.0,302.0,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,333.0,17644.0,1,2,1,4,Ostacolo_in_carreggiata,2018-10-08 17:45:00,2018-10-08 18:00:00,2018-10-08 18:15:00,2018-10-08 18:30:00,2018-10-08 18:45:00,2018-10-08 19:00:00,2018-10-08 19:15:00,2018-10-08 19:30:00,231.0138,237.6196,238.0555,234.3823,234.6660,225.2852,237.4857,225.6115,20.5212,22.1078,21.4170,20.3625,315.510,364.050,326.836,328.454,176.362,190.924,197.396,189.306,292.0,293.0,278.0,270.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,33.0,33.0,33.0,33.0,23.0,23.0,22.0,22.0,17.0,17.0,17.0,17.0,24.0,24.0,24.0,24.0
6,0.0,333.0,17792.0,1,2,1,19,Os

In [20]:
speeds = data.speeds_original()

caching resources/dataset/originals/speeds_train.csv.gz


In [7]:
def check_speed(df, key, km, datetime):
    return df[(df.KEY == key) & (df.KM == km) & (df.DATETIME_UTC == datetime)]

In [9]:
check_speed(speeds, key=0, km=333, datetime='2018-10-24 12:30:00')

,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2
5959614,0,2018-10-24 12:30:00,333,240.731711,22.643887,189.306,328.454,194,0_333


In [10]:
ev = data.events_original()

caching resources/dataset/originals/events_train.csv.gz


In [12]:
ev[ev.index == 18082.0]

,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KEY,KEY_2
18082,2018-10-24 13:13:03,2018-10-24 15:28:47,20.0,extended_accident,336,336,0.0,0_336


## Check discarded events

In [14]:
events_in_dataset = dataset.event_index.unique()
len(events_in_dataset)

5528

In [7]:
events_in_dataset

array([16789., 17038., 17088., ..., 30801., 30802., 30837.])

In [15]:
ev = data.events_original()
ev.head()

,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KEY,KEY_2
0,2018-07-19 07:34:46,2018-08-01 15:23:17,0.0,Gestione_viabilita,959,954,523.0,523_959
1,2018-07-25 12:20:06,2018-10-12 18:29:37,1.0,Pavimentazione,1814,1821,325.0,325_1814
2,2018-08-01 07:44:00,2018-08-01 16:18:58,2.0,Manutenzione_opere_in_verde,321,282,525.0,525_321
3,2018-08-01 08:14:30,2018-08-01 17:39:13,3.0,Opera_arte,713,712,517.0,517_713
4,2018-08-01 08:18:32,2018-08-01 17:38:27,3.0,Opera_arte,2542,2543,518.0,518_2542


In [17]:
discarded_ev = list(set(ev.index).difference(set(events_in_dataset)))
len(discarded_ev)

32480

In [19]:
ev.loc[[discarded_ev[0]]]

,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KEY,KEY_2
0,2018-07-19 07:34:46,2018-08-01 15:23:17,0.0,Gestione_viabilita,959,954,523.0,523_959


In [23]:
def check_involved_sensors_by_event(event_id, speeds_df):
    e = data.events_original().loc[event_id]
    return speeds_df[(speeds_df.KEY == e.KEY) & (speeds_df.KM >= e.KM_START) & (speeds_df.KM <= e.KM_END)]

In [29]:
for eid in discarded_ev:
    res = check_involved_sensors_by_event(eid, data.speeds_original())
    if res.shape[0] != 0:
        print(eid)
        break

2


In [30]:
ev.loc[[discarded_ev[2]]]

,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KEY,KEY_2
2,2018-08-01 07:44:00,2018-08-01 16:18:58,2.0,Manutenzione_opere_in_verde,321,282,525.0,525_321


In [31]:
check_involved_sensors_by_event(2, data.speeds_original())

,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2
477869,525,2018-09-01 06:15:00,305,238.032692,44.568372,59.866,279.914,26,525_305
477870,525,2018-09-01 09:00:00,305,233.610112,35.280787,74.428,378.612,89,525_305
477871,525,2018-09-01 20:45:00,305,222.673434,27.156433,174.744,299.330,53,525_305
477872,525,2018-09-01 21:15:00,305,212.814588,50.253969,77.664,313.892,34,525_305
477873,525,2018-09-01 23:45:00,305,200.632000,65.173474,90.608,249.172,5,525_305
477874,525,2018-09-02 07:45:00,305,252.408000,29.803460,215.194,321.982,18,525_305
477875,525,2018-09-02 14:45:00,305,171.921907,27.056109,74.428,228.138,43,525_305
477876,525,2018-09-02 17:45:00,305,232.543770,20.663552,165.036,292.858,148,525_305
477877,525,2018-09-02 23:30:00,305,261.660937,30.060699,215.194,338.162,32,525_305
477878,525,2018-09-03 06:00:00,305,245.494727,29.925545,189.306,330.072,33,525_305
